In [167]:
import re
import csv
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score
#from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from datetime import datetime
from elasticsearch import Elasticsearch #pip install Elasticsearch if not intalled yet
import json
import calendar
import matplotlib.pyplot as plt
%matplotlib inline

totalNaive = {'positif':0,'negatif':0,'netral':0,'data':0};

In [168]:
df = pd.read_csv('out.csv', delimiter=',', index_col = False, encoding = "ISO-8859-1" )
Posit = df.loc[df['Sentiment'] == 1].count()
TempPos = Posit['Sentiment']

Negat = df.loc[df['Sentiment'] == -1].count()
TempNeg =  Negat['Sentiment']

Neutr = df.loc[df['Sentiment'] == 0].count()
TempNeutr = Neutr['Sentiment']
# print(TempNeutr)

In [175]:
def initial():
    global totalNaive;
    totalNaive['positif'] = TempPos
    totalNaive['negatif'] = TempNeg
    totalNaive['netral'] = TempNeutr

    totalNaive['data'] = totalNaive['positif'] + totalNaive['negatif'] + totalNaive['netral'];

In [428]:
def probably(word,senti):
    global totalNaive;
    initial()
    #try:
    #    inWord = df['Mentions'].where(df['Sentiment'] == senti).str.contains(word).value_counts().values[1]
    #except:
    #    inWord = 0
    try:
        inWord = df['Mentions'].where(df['Sentiment'] == senti).str.contains(word).value_counts()[True]
    except:
        inWord = 0
    if(senti == 1):
        return (inWord/totalNaive['positif']) * (totalNaive['positif']/totalNaive['data']);
    elif(senti == -1):
        return (inWord/totalNaive['negatif']) * (totalNaive['negatif']/totalNaive['data']);
    elif(senti == 0):
        return (inWord/totalNaive['netral']) * (totalNaive['netral']/totalNaive['data']);

In [429]:
def counting(word):
    pPos    = probably(word,1);
    pNeg    = probably(word,-1);
    pNetral = probably(word,0);
    
    if(pPos >= pNeg):
        if(pPos >= pNetral):
            result = 1;
        else:
            if(pNeg >= pNetral):
                result = -1;
            else:
                result = 0;
    elif(pNeg >= pNetral):
        result = -1;
    else:
        result = 0;

    # print pPos;
    # print pNeg;
    # print pNetral;
    # print result;
    # print "===========";
    return result;

In [430]:
def play(sentence):
    sentence = sentence.lower()
    sentence=sentence.split(" ")
    sentiment = 0;
    for word in sentence:
        sentiment = sentiment + counting(word);
        print(word,counting(word),sentiment)
    
    if(sentiment > 0):
        return 1;
    elif(sentiment < 0):
        return -1;
    else:
        return 0;

In [431]:
play("ancaman gugatan dilayangkan partai berkarya jurubicara tkn ahmad basarah memancing kemarahan")

ancaman 0 0
gugatan 0 0
dilayangkan 0 0
partai -1 -1
berkarya -1 -2
jurubicara 0 -2
tkn -1 -3
ahmad -1 -4
basarah 0 -4
memancing 0 -4
kemarahan 0 -4


-1